In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.metrics import categorical_accuracy
from keras.optimizers import SGD
from keras.optimizers import adam
from keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ProjDataAccidentDUICategorized(Reduced).csv to ProjDataAccidentDUICategorized(Reduced).csv


In [ ]:
import pandas as pd
data = pd.read_csv('ProjDataAccidentDUICategorized.csv', delimiter=',', dtype="category") 

data.head()
#print(data.dtypes)

,STATE,PVH,PERNOTMVIT,PERMVIT,PERSONS,DAY,MONTH,DAYofWEEK,NHS,Rur_Urb,FUNC_SYS,RD_OWNER,ROUTE,SP_JUR,HARM_EV,MAN_COLL,RELJCT1,RELJCT2,TYP_INT,WRK_ZONE,REL_ROAD,LGT_COND,WEATHER1,CF1,CF2,CF3,FATALS,MinutesOfDay,Drunk Driver Involved
0,1,1,0,1,1,5,1,6,1,1,1,1,1,0,14,0,0,1,1,0,2,4,1,0,0,0,1,2,0
1,1,0,0,2,2,8,1,2,1,2,1,1,1,0,38,0,1,12,1,0,3,2,2,20,0,0,2,1,0
2,1,0,0,2,2,8,1,2,1,1,1,1,1,0,24,0,0,1,1,1,4,2,10,0,0,0,1,1,0
3,1,0,0,2,2,9,1,3,1,1,1,1,1,0,24,0,0,1,1,0,4,1,10,0,0,0,1,3,0
4,1,0,0,2,2,19,1,6,1,2,1,1,1,0,12,1,0,1,1,0,1,1,1,0,0,0,1,2,0


In [ ]:
import numpy as np
dataset = np.genfromtxt('ProjDataAccidentDUICategorized.csv', delimiter=",", skip_header = True) 
print(dataset.shape)

(33654, 29)


In [ ]:
#Splitting Inputs and Output
A=dataset[:,:28] #Inputs
B=dataset[:,28]  #Output

print(A.shape)
print(B.shape)

(33654, 28)
(33654,)


In [ ]:
#OneHotEncode Data

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(drop='first', sparse=False)
Xenc = enc.fit(A[:,:28]).transform(A[:,:29])
print(Xenc.shape)

(33654, 404)


In [ ]:
#Cross Validation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.metrics import categorical_accuracy
from keras.optimizers import SGD
from keras.optimizers import adam
from keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

X = Xenc
Y = dataset[ : , -1]

# Define 5 folds
kf = KFold(n_splits = 5, shuffle=True)
# Returns the number of splitting iterations in the cross-validator
kf.get_n_splits(X)
print(kf)

KFold(n_splits=5, random_state=None, shuffle=True)


In [ ]:
CrossValidAccuracy = np.arange(1, 1.0 + kf.get_n_splits(X))
count=0
split = 0
for train_indices, valid_indices in kf.split(X):
  print('')
  split += 1
  print('Split:', split)
  print('TRAIN:')
  print(train_indices.shape)
  print(train_indices) 
  print('VALID:')
  print(valid_indices.shape)
  print(valid_indices)
  XTRAIN, XVALID = X[train_indices], X[valid_indices]
  YTRAIN, YVALID = Y[train_indices], Y[valid_indices] 
  
  # Your code for training/evaluation goes here..
  model = Sequential()
  model.add(Dense(10, input_dim = 404, kernel_initializer='normal', activation='sigmoid'))
  #model.add(Dense(40,  kernel_initializer='normal', activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
  callback_a = ModelCheckpoint(filepath= 'project.hdf5', monitor='val_loss', save_best_only = True, save_weights_only = True, verbose = 1)
  callback_b = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 30, verbose=1)
  history = model.fit(XTRAIN, YTRAIN, batch_size=23000, epochs=2000, verbose=1, validation_data=(XVALID, YVALID), callbacks=[callback_a, callback_b]) 

  #Save Each Models Perfomance
  model.load_weights('project.hdf5')
  prediction = model.predict(XVALID)
  prediction = np.round(np.clip(model.predict(XVALID), 0, 1)) # binary
  Validation_Accuracy = accuracy_score(YVALID, prediction.round())
  CrossValidAccuracy[count] = (Validation_Accuracy * 100.0)
  count += 1

Streaming output truncated to the last 5000 lines.
1/2 [==============>...............] - ETA: 0s - loss: 0.4439 - binary_accuracy: 0.7900
Epoch 01174: val_loss improved from 0.45642 to 0.45642, saving model to project.hdf5
2/2 [==============================] - 0s 28ms/step - loss: 0.4439 - binary_accuracy: 0.7911 - val_loss: 0.4564 - val_binary_accuracy: 0.7849
Epoch 1175/2000
1/2 [==============>...............] - ETA: 0s - loss: 0.4447 - binary_accuracy: 0.7895
Epoch 01175: val_loss improved from 0.45642 to 0.45642, saving model to project.hdf5
2/2 [==============================] - 0s 29ms/step - loss: 0.4439 - binary_accuracy: 0.7910 - val_loss: 0.4564 - val_binary_accuracy: 0.7847
Epoch 1176/2000
1/2 [==============>...............] - ETA: 0s - loss: 0.4426 - binary_accuracy: 0.7913
Epoch 01176: val_loss improved from 0.45642 to 0.45641, saving model to project.hdf5
2/2 [==============================] - 0s 28ms/step - loss: 0.4439 - binary_accuracy: 0.7912 - val_loss: 0.4564 - 

In [ ]:
import statistics 
index = np.arange(kf.get_n_splits(X))
#print(CrossValidAccuracy)

for i in index:
  CrossValidAccuracy[i] = int((CrossValidAccuracy[i] * 100) + 0.5) / 100.0 #Convert 2 decimal and round up

print("Cross-Valid Accuracy: " +str(CrossValidAccuracy))
print("Mean: %.2f%%" %CrossValidAccuracy.mean())
print("Max: %.2f%%" % max(CrossValidAccuracy))
print("Min: %.2f%%" %min(CrossValidAccuracy))
StandardD = int((statistics.stdev(CrossValidAccuracy)* 100) + 0.5) / 100.0
print("Standard Deviation: " + str(StandardD))


Cross-Valid Accuracy: [77.91 79.27 78.64 78.58 77.61]
Mean: 78.40%
Max: 79.27%
Min: 77.61%
Standard Deviation: 0.65


In [ ]:

#table = pd.DataFrame([CrossValidAccuracy], columns="Accuracy")
print("| Split  |  Accuracy |")
print("======================")
for i in index:
  print("    " + str(i+1)+ "       " + str(CrossValidAccuracy[i-1]) + "%    " )

| Split  |  Accuracy |
    1       77.61%    
    2       77.91%    
    3       79.27%    
    4       78.64%    
    5       78.58%    


In [ ]:
scores = cross_val_score(model, X, Y, cv=5)
print (“Cross-validated scores:”, scores)